In [5]:
%pip install requests argparse pandas
%matplotlib inline

import requests, datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------- -------------------------------- 2.1/11.0 MB 11.7 MB/s eta 0:00:01
   ---------------- ----------------------- 4.5/11.0 MB 11.7 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 11.6 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.0 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 11.4 MB/s  0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [argparse]
   -------------------- ------------------- 2/4 [tzdata]
   ------------------------------ --------- 3/4 [pandas]
   ------------------------------ --------- 3/4 [pandas]
   -------

ModuleNotFoundError: No module named 'ipywidgets'

In [ ]:
API = "https://api.blockchain.info/charts/hash-rate"

UNIT_MULT = {
    "H/s": 1,
    "KH/s": 1e3,
    "MH/s": 1e6,
    "GH/s": 1e9,
    "TH/s": 1e12,
    "PH/s": 1e15,
    "EH/s": 1e18,
}

def fetch_df(timespan=None, start=None, end=None):
    params = {"format": "json"}
    if timespan:
        params["timespan"] = timespan
    if start:
        params["start"] = start
    if end:
        params["end"] = end
    r = requests.get(API, params=params, timeout=15)
    r.raise_for_status()
    js = r.json()
    vals = js.get("values", [])
    if not vals:
        return pd.DataFrame(columns=["datetime","hashrate"]).set_index("datetime")
    unit = js.get("unit", "")
    mult = UNIT_MULT.get(unit, 1)
    df = pd.DataFrame(vals)
    df["datetime"] = pd.to_datetime(df["x"], unit="s")
    df["hashrate_Hs"] = df["y"].astype(float) * mult
    df = df.set_index("datetime").sort_index()
    return df[["hashrate_Hs"]]

def plot_hashrate(df, ax=None, show=True, title="Bitcoin Network Hash Rate"):
    if ax is None:
        fig, ax = plt.subplots(figsize=(10,4))
    else:
        fig = ax.figure
    ax.clear()
    ax.plot(df.index, df["hashrate_Hs"] / 1e18, linewidth=1)
    ax.set_ylabel("Hash rate (EH/s)")
    ax.set_title(title)
    ax.grid(alpha=0.3)
    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(mdates.AutoDateLocator()))
    fig.tight_layout()
    if show:
        display(fig)

In [6]:
range_select = widgets.Dropdown(options=["1y","all","custom"], value="1y", description="Range:")
start_picker = widgets.DatePicker(description="Start", value=None)
end_picker = widgets.DatePicker(description="End", value=None)
update_btn = widgets.Button(description="Update")
out = widgets.Output(layout={"border":"1px solid #ddd"})

def on_range_change(change):
    if change["new"] == "custom":
        start_picker.disabled = False
        end_picker.disabled = False
    else:
        start_picker.disabled = True
        end_picker.disabled = True

range_select.observe(on_range_change, names="value")
start_picker.disabled = True
end_picker.disabled = True

def on_update(_):
    with out:
        clear_output(wait=True)
        try:
            r = range_select.value
            if r == "1y":
                df = fetch_df(timespan="365days")
                title = "Last 1 year"
            elif r == "all":
                df = fetch_df(start="2009-01-03", end=datetime.date.today().isoformat())
                title = "All time (2009 → today)"
            else:
                if not (start_picker.value and end_picker.value):
                    print("Please pick start and end dates.")
                    return
                df = fetch_df(start=start_picker.value.isoformat(), end=end_picker.value.isoformat())
                title = f"Custom: {start_picker.value} → {end_picker.value}"
            if df.empty:
                print("No data returned for the selected range.")
                return
            plot_hashrate(df, title=title)
        except Exception as e:
            print("Error:", e)

update_btn.on_click(on_update)

controls = widgets.HBox([range_select, start_picker, end_picker, update_btn])
display(controls, out)
# auto-run once
on_update(None)

NameError: name 'widgets' is not defined